# Tugas Pemrograman 1 - Pengantar Kecerdasan Buatan
Kelompok 6:
1. Hilman Taris Muttaqin
2. Herjanto Janawisuta

Definisi Tugas:
Lakukan analisis dan desain algoritma Genetic Algorithm (GA) serta mengimplementasikannya ke dalam suatu program komputer untuk mencari nilai 𝑥 dan 𝑦 sehingga diperoleh nilai minimum dari fungsi:
>ℎ(𝑥,𝑦)=(cos𝑥+sin𝑦)2𝑥2+𝑦2

dengan domain (batas nilai) untuk 𝑥 dan 𝑦:

>−5≤𝑥≤5 dan −5≤𝑦≤5

Hal yang harus Anda analisis dan desain
- Desain kromosom dan metode dekode-nya
- Ukuran populasi
- Metode pemilihan orangtua
- Metode operasi genetik (pindah silang dan mutasi)
- Probabilitas operasi genetik (𝑃𝑐 dan 𝑃𝑚)
- Metode pergantian generasi (seleksi survivor)
- Kriteria penghentian evolusi (loop)
> Catatan: Poin-poin di atas harus ada di dalam Laporan Tugas!

Proses yang harus Anda bangun/implementasikan ke dalam baris-baris program:
- Dekode kromosom
- Perhitungan fitness
- Pemilihan orangtua
- Crossover (pindah silang)
- Mutasi
- Pergantian Generasi
>Catatan: Proses-proses di atas harus dibangun tanpa menggunakan Library!

**Output Program**
Dengan masalah yang didefinisikan di atas, output dari program Anda:
- kromosom terbaik, dan
- nilai 𝒙 dan 𝒚 hasil dekode kromosom terbaik tersebut.

## Inisialisasi data constraint dan import library yang dibutuhkan
library yang digunakan merupakan library bawaan dari python, yaitu:
- random
- math

data constraint yang didefinisikan adalah limit atas dan limit bawah x dan y.

> Untuk mempermudah pembacaan codingan, terdapat data constant yang bisa digunakan

In [104]:
# Import library random dan operasi math sin dan cos
import random
from math import sin, cos

# Inisialisasi nilai domain dari x dan y, buat variable untuk limit x dan y
limit_x = [-5, 5] # -5 >= x >= 5
limit_y = [-5, 5] # -5 >= y >= 5

# constant untuk limit
LIMIT_BAWAH = 0
LIMIT_ATAS = 1

chromosome_length = 10
max_population = 10
generation_count = 100

# domain constant
X = 0
Y = 1

# population, parents, and offsprings struct
# population = [
#     {
#         'chromosome': [1, 2, 3, 4, 3, 1],
#         'fitness': 2.32399403989493
#     }
# ]

# population constant
CHROMOSOME = 'chromosome'
FITNESS = 'fitness'

## Inisialisasi fungsi
Disini adalah bagian inisialisasi fungsi yang digunakan dalam pengerjaan tugas algoritma genetika.
Fungsi-fungsi terkait **algortima genetika** didefinisikan disini:
1. Generate Populasi
2. Hitung nilai fitness dari setiap kromosom
3. Pencarian orangtua
4. Elisitasi
5. Crossover
6. Mutasi
7. Seleksi survivor

### Generate Populasi
Populasi akan di generate menggunakan tipe data integer untuk masing-masing gen pada suatu kromosom (Integer encoding).

In [105]:
def generate_population(population, kromosom):
    # return random integer k gen in single chromosome and generate p population of chromosome
    return [{CHROMOSOME: [random.randint(0, 9) for _ in range(kromosom)], FITNESS: 0} for _ in range(population)]

# Uncomment this for testing purposes
# print(generate_population(10, 6))

#### Decode Populasi
Representasi data gen yang digunakan adalah dengan tipe data integer. Sehingga butuh rumus untuk digunakan sebagai fungsi decoding dari 3 gen pada kromosom menjadi floating point sesuai dengan limit atas dan limit bawah x dan y.

> 𝑥= 𝑟_𝑚𝑖𝑛 + ((𝑟_𝑚𝑎𝑥 − 𝑟_𝑚𝑖𝑛) / (Σ_(𝑖=1)^𝑁 9∗10^(−𝑖))) (𝑔_1∗10^(−1)+𝑔_1∗10^(−2)+…+𝑔_𝑁∗10^(−𝑁) )

In [106]:
def get_domain_from_kromosom(kromosom):
    # return split gens in chromosome
    # e.g. [1, 2, 4, 2, 3, 3]
    # result = [[1, 2, 4], [2, 3, 3]]
    pembagi = int(chromosome_length / 2)
    return [kromosom[:pembagi], kromosom[pembagi:]]

# Uncomment this for testing purposes
# dummy_get_domain_from_kromosom = [random.randint(0, 9) for _ in range(6)]
# domain_get_domain_from_kromosom = get_domain_from_kromosom(dummy_get_domain_from_kromosom)
# print("kromosom = ", dummy_get_domain_from_kromosom)
# print("domain X = ", domain_get_domain_from_kromosom[X])
# print("domain Y = ", domain_get_domain_from_kromosom[Y])

In [107]:
def decode(kromosom, limit):
    # return decode result from x or y in single chromosome
    # result in floating point data type
    pengali = 0
    pembagi = 0

    for i in range(len(kromosom)):
        gen = kromosom[i]
        pengali += gen * 10**-(i+1)
        pembagi += 9 * 10**-(i+1)
    
    return limit[LIMIT_BAWAH] + (((limit[LIMIT_ATAS] - limit[LIMIT_BAWAH]) / pembagi) * pengali)

# Uncomment this for testing purposes
# run get_domain_from_kromosom() function before test this function 

# dummy_get_domain_from_kromosom = [random.randint(0, 9) for _ in range(chromosome_length)]
# domain_get_domain_from_kromosom = get_domain_from_kromosom(dummy_get_domain_from_kromosom)
# print("kromosom = ", dummy_get_domain_from_kromosom)
# print("domain X = ", domain_get_domain_from_kromosom[X])
# print("domain Y = ", domain_get_domain_from_kromosom[Y])
# print("decode Y = ", decode(domain_get_domain_from_kromosom[X], limit_x))
# print("decode Y = ", decode(domain_get_domain_from_kromosom[Y], limit_y))

### Fungsi fitness
Fitness merupakan nilai kualitas dari suatu kromosom. Semakin tinggi nilai fitness (fittest) maka kemungkinan bertahan pada generasi selanjutnya semakin besar, sebaliknya jika semakin kecil nilai fitness maka akan semakin besar juga kromosom tidak akan bertahan di generasi selanjutnya. 

#### Fungsi objektif
Mengevaluasi kromosom. 1 solusi bisa saja memiliki beberapa fungsi objektif.

In [108]:
def h(x, y):
    # this function called objective function
    # this function provided by this task in task description 
    return ((cos(x) + sin(y))**2) / (x**2 + y**2)

#### Fungsi fitness
Karena pada soal harus **meminimasi** fungsi h(n), maka rumus yang digunakan adalah

> f = 1 / h + a

In [109]:
def fitness(x, y):
    # because the task is search for minimum value
    # formula used in this function is 1 / h + a
    # addition of 0.01 is used to avoid dividing by 0
    if (x >= limit_x[LIMIT_BAWAH] and x <= limit_x[LIMIT_ATAS]) and (y >= limit_y[LIMIT_BAWAH] and y <= limit_y[LIMIT_ATAS]):
        return 1 / h(x, y) + 0.01
    else:
        return 0

# dummy_get_domain_from_kromosom = [random.randint(0, 9) for _ in range(chromosome_length)]
# domain_get_domain_from_kromosom = get_domain_from_kromosom(dummy_get_domain_from_kromosom)
# print("kromosom = ", dummy_get_domain_from_kromosom)
# print("domain X = ", domain_get_domain_from_kromosom[X])
# print("domain Y = ", domain_get_domain_from_kromosom[Y])
# x = decode(domain_get_domain_from_kromosom[X], limit_x)
# y = decode(domain_get_domain_from_kromosom[Y], limit_y)
# print("decode X = ", x)
# print("decode Y = ", y)
# print(fitness(x, y))

### Seleksi orang tua
Seleksi orang tua ini merupakan proses untuk memilih suatu individu untuk bisa membuat keturunan pada generasi selanjutnya. Individu dipilih berdasarkan nilai fitnessnya.

Fungsi yang digunakan untuk memilih orang tua adalah **stokastik roulette wheels**.

> **Pilih 2 orang tua saat algoritma genetik dijalankan**

In [110]:
def stochastic_roulette_wheel(population):
    max_fitness_chromosome = max(population, key = lambda x: x[FITNESS])
    while True:
        indv = random.uniform(0, 1) * max_population
        indv_chromosome = population[int(indv)]
        r = random.uniform(0, 1)
        if r < indv_chromosome[FITNESS] / max_fitness_chromosome[FITNESS]:
            return population[int(indv)]

### Crossover dan Mutasi
Proses mengacak dan menyatukan gen gen pada tiap parent untuk mendapatkan mutasi pada keturunan baru. Tetapi crossover tidak selalu dapat terjadi, probability crossover diantara 60% - 70%. Fungsi crossover yang digunakan adalah **single point crossover** karena panjang dari kromosomnya tidak begitu panjang.

#### Crossover

In [111]:
CROSSOVER_PROBABILITY = 0.7

In [112]:
def single_point_crossover(parent1, parent2):

    r = random.uniform(0, 1)

    if r < CROSSOVER_PROBABILITY:
        random_position = random.randint(0, chromosome_length - 1)

        offspring1_chromosome = parent1[CHROMOSOME][:random_position] + parent2[CHROMOSOME][random_position:]
        offspring2_chromosome = parent2[CHROMOSOME][:random_position] + parent1[CHROMOSOME][random_position:]

        offspring1 = {CHROMOSOME: offspring1_chromosome, FITNESS: 0}
        offspring2 = {CHROMOSOME: offspring2_chromosome, FITNESS: 0}

        result = [offspring1, offspring2]
    else:
        result = [parent1, parent2]

    # use this result for testing purposes
    # result = {
    #     'random position': random_position,
    #     'offspring 1': offspring1,
    #     'offspring 2': offspring2
    # }

    return result

# Uncomment this for testing purposes
# parent1 = [1, 2, 3, 2, 4, 5]
# parent2 = [2, 4, 1, 3, 1, 3]
# print(single_point_crossover(parent1, parent2))

#### Mutation

In [113]:
MUTATION_PROBABILITY = 0.01

In [114]:
def mutation(offspring_chromosome):
    
    for index, gen in enumerate(offspring_chromosome):
        r = random.uniform(0, 1)
        if r < MUTATION_PROBABILITY:
            new_gen = random.randint(0, 9)
            offspring_chromosome[index] = new_gen
    
    return offspring_chromosome

### Seleksi Survivor
Terdapat 2 cara untuk seleksi siapa yang bisa bertahan pada generasi selanjutnya, yaitu:
- Generational Replacement with Elitism mechanism
- Steady-State, pada contoh tugas ini menggunakan **Local Fitness-based selection**

#### Steady-State Local Fitness-based Selection

In [115]:
# max population replacement in this case is 4
POPULATION_REPLACEMENT = 2

def steady_state(population, parents, offsprings):
    new_population = population


    # combine parents and offsprings
    next_gen_candidate = parents + offsprings

    next_gen = []
    for i in range(POPULATION_REPLACEMENT):
        if len(next_gen_candidate) != 0:
            max_fitness_indv = max(next_gen_candidate, key = lambda x: x[FITNESS])
            next_gen.append(max_fitness_indv)
            next_gen_candidate.remove(max_fitness_indv)

    remaining_generations = 0
    for index, pop in enumerate(new_population):
        if remaining_generations <= 1:
            for parent in parents:
                if pop == parent and len(next_gen) != 0:
                    new_population[index] = next_gen[remaining_generations]
                    remaining_generations += 1


    return new_population

#### General Replacement (Elisitisme)

In [116]:
# if the population size is even, elisitism the population twice
# if the popoulation size is odd, elisitism the population in single value
def elisitism(population):

    if len(population) % 2 != 0:
        return [max(population, key = lambda x: x[FITNESS])]
    else:
        new_population = []
        while len(new_population) < 2:
            max_fitness_chromosome = max(population, key = lambda x: x[FITNESS])
            new_population.append(max_fitness_chromosome)
        return new_population

## Playground
Menyatukan seluruh inisialisasi fungsi menjadi algoritma genetika

### Menggunakan Steady-State

In [117]:
# generate initial chromosom
populations = generate_population(max_population, chromosome_length)

best_chromosome = max(populations, key = lambda x: x[FITNESS])
print("Kromosom terbaik : ", best_chromosome[CHROMOSOME])
print("Fitness value = ", best_chromosome[FITNESS])

for i in range(generation_count):

    # hitung nilai fitness
    for chromosome in populations:
        domain = get_domain_from_kromosom(chromosome[CHROMOSOME])
        current_x = decode(domain[X], limit_x)
        current_y = decode(domain[Y], limit_y)
        chromosome[FITNESS] = fitness(current_x, current_y)
    
    # seleksi orang tua
    parents = []
    parents.append(stochastic_roulette_wheel(populations))
    parent2 = stochastic_roulette_wheel(populations)
    while parent2 == parents[0]:
        parent2 = stochastic_roulette_wheel(populations)
    parents.append(parent2)

    # crossover
    offsprings = single_point_crossover(parents[0], parents[1])

    # mutation
    for offspring in offsprings:
        offspring[CHROMOSOME] = mutation(offspring[CHROMOSOME])
        domain = get_domain_from_kromosom(offspring[CHROMOSOME])
        current_x = decode(domain[X], limit_x)
        current_y = decode(domain[Y], limit_y)
        offspring[FITNESS] = fitness(current_x, current_y)

    # survivor selection
    populations = steady_state(populations, parents, offsprings)
    
    print(f"\n\n=================== gen {i} ===================")

    best_chromosome = max(populations, key = lambda x: x[FITNESS])
    print("Kromosom terbaik : ", best_chromosome[CHROMOSOME])
    print("Fitness value = ", best_chromosome[FITNESS])

print(f"\n\n================== FINAL GENERATION ==================")
for chromosome in populations:
    domain = get_domain_from_kromosom(chromosome[CHROMOSOME])
    current_x = decode(domain[X], limit_x)
    current_y = decode(domain[Y], limit_y)
    print(f"Kromosom: {chromosome[CHROMOSOME]}")
    print(f"x: {current_x}")
    print(f"y: {current_y}")
    print(f"Fitness: {chromosome[FITNESS]}")
    print("=============")

Kromosom terbaik :  [3, 6, 3, 5, 7, 5, 5, 5, 0, 6]
Fitness value =  0


=================== gen 0 ===================
Kromosom terbaik :  [4, 0, 1, 0, 2, 8, 9, 6, 5, 4]
Fitness value =  488.57985699547805


=================== gen 1 ===================
Kromosom terbaik :  [4, 0, 1, 0, 2, 8, 9, 6, 5, 4]
Fitness value =  488.57985699547805


=================== gen 2 ===================
Kromosom terbaik :  [4, 0, 1, 0, 2, 8, 9, 6, 5, 4]
Fitness value =  488.57985699547805


=================== gen 3 ===================
Kromosom terbaik :  [4, 0, 1, 0, 2, 8, 9, 6, 5, 4]
Fitness value =  488.57985699547805


=================== gen 4 ===================
Kromosom terbaik :  [4, 0, 1, 0, 2, 8, 9, 6, 5, 4]
Fitness value =  488.57985699547805


=================== gen 5 ===================
Kromosom terbaik :  [4, 0, 1, 0, 2, 8, 9, 6, 5, 4]
Fitness value =  488.57985699547805


=================== gen 6 ===================
Kromosom terbaik :  [4, 5, 1, 0, 2, 8, 9, 6, 5, 4]
Fitness value =  722.

### Single test playground without generation loop

In [118]:
# generate initial chromosom
populations = generate_population(max_population, chromosome_length)

# hitung nilai fitness
print("============== HITUNG NILAI FITNESS ==============")
for chromosome in populations:
  domain = get_domain_from_kromosom(chromosome[CHROMOSOME])
  current_x = decode(domain[X], limit_x)
  current_y = decode(domain[Y], limit_y)
  chromosome[FITNESS] = fitness(current_x, current_y)
  print(f"Kromosom: {chromosome[CHROMOSOME]}")
  print(f"x: {current_x}")
  print(f"y: {current_y}")
  print(f"Fitness: {chromosome[FITNESS]}")
  print("=============")
  

# seleksi orang tua
parents = []
parents.append(stochastic_roulette_wheel(populations))
parent2 = stochastic_roulette_wheel(populations)
while parent2 == parents[0]:
  parent2 = stochastic_roulette_wheel(populations)
parents.append(parent2)

print("\n\n============== PARENT SELECTION ==============")
for index, parent in enumerate(parents):
  domain = get_domain_from_kromosom(parent[CHROMOSOME])
  current_x = decode(domain[X], limit_x)
  current_y = decode(domain[Y], limit_y)
  print(f"Kromosom: {parent[CHROMOSOME]}")
  print(f"x: {current_x}")
  print(f"y: {current_y}")
  print(f"Fitness: {parent[FITNESS]}")
  print("=============")

# crossover
offsprings = single_point_crossover(parents[0], parents[1])

print("\n\n============== CROSSOVER ==============")
for index, offspring in enumerate(offsprings):
  domain = get_domain_from_kromosom(offspring[CHROMOSOME])
  current_x = decode(domain[X], limit_x)
  current_y = decode(domain[Y], limit_y)
  print(f"Kromosom: {offspring[CHROMOSOME]}")
  print(f"x: {current_x}")
  print(f"y: {current_y}")
  print(f"Fitness: {offspring[FITNESS]}")
  print("=============")

# mutation
for offspring in offsprings:
  offspring[CHROMOSOME] = mutation(offspring[CHROMOSOME])
  domain = get_domain_from_kromosom(offspring[CHROMOSOME])
  current_x = decode(domain[X], limit_x)
  current_y = decode(domain[Y], limit_y)
  offspring[FITNESS] = fitness(current_x, current_y)

print("\n\n============== MUTASI ==============")
for index, offspring in enumerate(offsprings):
  domain = get_domain_from_kromosom(offspring[CHROMOSOME])
  current_x = decode(domain[X], limit_x)
  current_y = decode(domain[Y], limit_y)
  print(f"Kromosom: {offspring[CHROMOSOME]}")
  print(f"x: {current_x}")
  print(f"y: {current_y}")
  print(f"Fitness: {offspring[FITNESS]}")
  print("=============")

# survivor selection
populations = steady_state(populations, parents, offsprings)

print("\n\n============== SURVIVOR SELECTION ==============")
for chromosome in populations:
  domain = get_domain_from_kromosom(chromosome[CHROMOSOME])
  current_x = decode(domain[X], limit_x)
  current_y = decode(domain[Y], limit_y)
  print(f"Kromosom: {chromosome[CHROMOSOME]}")
  print(f"x: {current_x}")
  print(f"y: {current_y}")
  print(f"Fitness: {chromosome[FITNESS]}")
  print("=============")
    
best_chromosome = max(populations, key = lambda x: x[FITNESS])
print("\n\nKromosom terbaik : ", best_chromosome[CHROMOSOME])
print("Fitness value = ", best_chromosome[FITNESS])

============== HITUNG NILAI FITNESS ==============
Kromosom: [0, 9, 2, 5, 6, 9, 1, 5, 4, 6]
x: -4.074390743907439
y: 4.154691546915467
Fitness: 16.248313680216018
Kromosom: [1, 5, 8, 7, 9, 0, 3, 2, 2, 1]
x: -3.4120841208412083
y: -4.677896778967789
Fitness: 26208.400131198545
Kromosom: [3, 1, 0, 9, 8, 8, 4, 2, 8, 0]
x: -1.8901689016890164
y: 3.4280842808428087
Fitness: 43.070469523900876
Kromosom: [1, 9, 0, 5, 4, 0, 2, 7, 0, 8]
x: -3.094580945809458
y: -4.72919729197292
Fitness: 34401196.08997479
Kromosom: [2, 3, 1, 5, 9, 3, 3, 9, 0, 0]
x: -2.684076840768408
y: -1.6099660996609964
Fitness: 2.7339993594332315
Kromosom: [9, 3, 0, 4, 1, 2, 3, 7, 5, 7]
x: 4.304193041930418
y: -2.624276242762428
Fitness: 31.98481694959338
Kromosom: [6, 7, 8, 2, 1, 1, 8, 7, 7, 0]
x: 1.7821678216782164
y: -3.1229812298122983
Fitness: 247.82855306156776
Kromosom: [4, 6, 7, 5, 5, 3, 2, 7, 7, 8]
x: -0.32445324453244595
y: -1.7221672216722164
Fitness: 1850.3589988521428
Kromosom: [2, 7, 6, 8, 0, 6, 9, 1, 2, 3]
x:

### Menggunakan general replacement

In [119]:
populations = generate_population(max_population, chromosome_length)

for i in range(generation_count):

    # hitung nilai fitness
    for chromosome in populations:
        domain = get_domain_from_kromosom(chromosome[CHROMOSOME])
        current_x = decode(domain[X], limit_x)
        current_y = decode(domain[Y], limit_y)
        chromosome[FITNESS] = fitness(current_x, current_y)

    # elisitism
    new_population = elisitism(populations)

    while len(new_population) < max_population:

        # seleksi orang tua
        parents = []
        parents.append(stochastic_roulette_wheel(populations))
        parent2 = stochastic_roulette_wheel(populations)
        while parent2 == parents[0]:
            parent2 = stochastic_roulette_wheel(populations)
        parents.append(parent2)

        # crossover
        offsprings = single_point_crossover(parents[0], parents[1])

        # mutation
        for offspring in offsprings:
            offspring[CHROMOSOME] = mutation(offspring[CHROMOSOME])
            domain = get_domain_from_kromosom(offspring[CHROMOSOME])
            current_x = decode(domain[X], limit_x)
            current_y = decode(domain[Y], limit_y)
            offspring[FITNESS] = fitness(current_x, current_y)
        
        new_population += offsprings

    populations = new_population

    print(f"\n\n=================== gen {i} ===================")

    best_chromosome = max(populations, key = lambda x: x[FITNESS])
    print("Kromosom terbaik : ", best_chromosome[CHROMOSOME])
    print("Fitness value = ", best_chromosome[FITNESS])

print(f"\n\n================== FINAL GENERATION ==================")
for chromosome in populations:
    domain = get_domain_from_kromosom(chromosome[CHROMOSOME])
    current_x = decode(domain[X], limit_x)
    current_y = decode(domain[Y], limit_y)
    print(f"Kromosom: {chromosome[CHROMOSOME]}")
    print(f"x: {current_x}")
    print(f"y: {current_y}")
    print(f"Fitness: {chromosome[FITNESS]}")
    print("=============")



=================== gen 0 ===================
Kromosom terbaik :  [1, 4, 5, 6, 1, 7, 1, 8, 1, 2]
Fitness value =  1704.6706407349293


=================== gen 1 ===================
Kromosom terbaik :  [1, 4, 5, 6, 1, 7, 1, 8, 1, 2]
Fitness value =  1704.6706407349293


=================== gen 2 ===================
Kromosom terbaik :  [1, 4, 5, 6, 1, 7, 1, 8, 1, 2]
Fitness value =  1704.6706407349293


=================== gen 3 ===================
Kromosom terbaik :  [1, 4, 5, 6, 1, 7, 1, 8, 1, 2]
Fitness value =  1704.6706407349293


=================== gen 4 ===================
Kromosom terbaik :  [1, 4, 5, 6, 1, 7, 1, 8, 1, 2]
Fitness value =  1704.6706407349293


=================== gen 5 ===================
Kromosom terbaik :  [1, 4, 5, 6, 1, 7, 1, 8, 1, 2]
Fitness value =  1704.6706407349293


=================== gen 6 ===================
Kromosom terbaik :  [1, 4, 5, 6, 1, 7, 1, 8, 1, 2]
Fitness value =  1704.6706407349293


=================== gen 7 ===================
Kromoso